In [4]:
import numpy as np

# Orthogonal Matching Pursuit (OMP) and Sequential Pursuit (SP)

## Problem Formulation

Given a measurement vector ($\mathbf{b} \in \mathbb{C}^{m} $) and a dictionary matrix ($\mathbf{A} \in \mathbb{C}^{m \times n}$), we aim to find a sparse solution ($\mathbf{x} \in \mathbb{C}^{n}$) such that:

$$
\mathbf{b} = \mathbf{A} \mathbf{x} + \mathbf{e},
$$

where $(\mathbf{e})$ is a small error term.




## Orthogonal Matching Pursuit (OMP)

OMP iteratively selects the column from $( \mathbf{A} )$ that has the highest correlation with the residual.

1. **Initialize:** Set residual $( \mathbf{r} = \mathbf{b} )$, support set $( S = \emptyset )$, and estimated sparse vector $( \mathbf{x} = \mathbf{0} )$.
2. **Iteration:** Repeat for $( k = 1, \dots, s )$:
   - Compute the correlation:

     $$
     j_k = \arg\max_j \left| \mathbf{A}_j^H \mathbf{r} \right|
     $$

   - Update support set:

     $$
     S = S \cup \{ j_k \}
     $$

   - Solve the least-squares problem:

     $$
     \mathbf{x}_S = \arg\min_{\mathbf{z}} \|\mathbf{A}_S \mathbf{z} - \mathbf{b} \|_2
     $$

   - Update residual:

     $$
     \mathbf{r} = \mathbf{b} - \mathbf{A}_S \mathbf{x}_S
     $$

3. **Return:** Sparse solution $( \mathbf{x} )$ with nonzero values in $( S )$.



In [12]:
def orthogonal_matching_pursuit(A, b, sparsity):
    """ Orthogonal Matching Pursuit (OMP) for complex-valued data. """
    m, n = A.shape
    residual = b.copy()
    support = []
    x = np.zeros(n, dtype=np.complex128)
    
    for _ in range(sparsity):
        correlations = A.conj().T @ residual  # Compute correlations
        idx = np.argmax(np.abs(correlations))  # Find the index of the max correlation
        support.append(idx)
        
        A_restricted = A[:, support]  # Restricted dictionary
        x_restricted = np.linalg.lstsq(A_restricted, b, rcond=None)[0]  # Solve least squares
        
        residual = b - A_restricted @ x_restricted  # Update residual
    
    x[support] = x_restricted  # Assign values to x
    return x

## Sequential Pursuit (SP)

SP selects columns based on maximum correlation but updates $( \mathbf{x} )$ at each step.

1. **Initialize:** Set residual $( \mathbf{r} = \mathbf{b} )$, support set $( S = \emptyset )$, and estimated sparse vector $( \mathbf{x} = \mathbf{0} )$.
2. **Iteration:** Repeat for $( k = 1, \dots, s )$:
   - Compute the correlation:

     $$
     j_k = \arg\max_j \left| \mathbf{A}_j^H \mathbf{r} \right|
     $$

   - Update support set:

     $$
     S = S \cup \{ j_k \}
     $$

   - Update sparse solution:

     $$
     \mathbf{x}_S = \mathbf{A}_S^\dagger \mathbf{b}
     $$

   - Update residual:

     $$
     \mathbf{r} = \mathbf{b} - \mathbf{A} \mathbf{x}
     $$

3. **Return:** Sparse solution $( \mathbf{x} )$.

In [13]:
def sequential_pursuit(A, b, sparsity):
    """ Sequential Pursuit (SP) for complex-valued data. """
    m, n = A.shape
    residual = b.copy()
    support = []
    x = np.zeros(n, dtype=np.complex128)
    
    for _ in range(sparsity):
        correlations = A.conj().T @ residual  # Compute correlations
        idx = np.argmax(np.abs(correlations))  # Find the index of the max correlation
        support.append(idx)
        x_subset = np.zeros(n, dtype=np.complex128)
        x_subset[support] = np.linalg.pinv(A[:, support]) @ b  # Compute solution using pseudo-inverse
        residual = b - A @ x_subset  # Update residual
    
    x = x_subset  # Assign values to x
    return x

In [15]:
A.shape


(20, 40)

In [14]:
# Example usage:
m, n, sparsity = 20, 40, 5
A = np.random.randn(m, n) + 1j * np.random.randn(m, n)  # Complex dictionary matrix
x_true = np.zeros(n, dtype=np.complex128)
nonzero_indices = np.random.choice(n, sparsity, replace=False)
x_true[nonzero_indices] = np.random.randn(sparsity) + 1j * np.random.randn(sparsity)
b = A @ x_true  # Generate measurements

x_omp = orthogonal_matching_pursuit(A, b, sparsity)
x_sp = sequential_pursuit(A, b, sparsity)

print("OMP Solution:", x_omp)
print("SP Solution:", x_sp)

OMP Solution: [ 0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.12568611+1.41237675j
  0.        +0.j         -0.40252193+0.46837757j  0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.81676521-0.28333119j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j         -0.28219731-1.06936979j  0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
 -0.17481321+1.54991436j]
SP Solution: [ 0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j     